In [30]:
import os
from osgeo import gdal
import numpy as np
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromFile

class TiffProcessor:
    def __init__(self, filepath):
        self.dataset = gdal.Open(filepath)
        if self.dataset is None:
            raise Exception("Could not open the dataset")
        self.x_size = self.dataset.RasterXSize
        self.y_size = self.dataset.RasterYSize
        self.tile_size = 500

    def create_tile(self, x_offset, y_offset, channel, output_filename):
        band = self.dataset.GetRasterBand(channel)
        tile_data = band.ReadAsArray(x_offset, y_offset, self.tile_size, self.tile_size)
        if tile_data is None:
            return
        
        # Normalizing the tile_data
        min_val = tile_data.min()
        max_val = tile_data.max()
        tile_data = (tile_data - min_val) / (max_val - min_val)
        
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(output_filename, self.tile_size, self.tile_size, 1, gdal.GDT_Float32)
        output_band = output.GetRasterBand(1)
        output_band.WriteArray(tile_data)
        output = None  # Close the file

    def process_tiles(self):
        output_directory = f"tile-{self.dataset.GetDescription().split('/')[-1].split('.')[0]}"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
    
        for y_offset in range(0, self.y_size, self.tile_size):
            for x_offset in range(0, self.x_size, self.tile_size):
                output_filename = os.path.join(output_directory, f"x{x_offset}_y{y_offset}.tif")
                self.create_tile(x_offset, y_offset, 1, output_filename)
                    

if __name__ == '__main__':
    # processor = TiffProcessor('gis-raw/n00_e011_1arc_v3.tif')
    # processor.process_tiles()
    feature_name = r"n00_e011_1arc_v3"
    feature_path = r"gis-raw/"
    feature_file = r"{feature_path}{feature_name}.tif"
    features = imageChipsFromFile(feature_file, x_size=7, y_size=7)
    print("Shape of features:", features.shape)
    feature_name = f"{label_name}.npy"
    
    # np.save(feature_name, features)

    built_features = features



AttributeError: 'NoneType' object has no attribute 'ReadAsArray'